In [1]:
import pandas as pd
import xmlrpc.client

In [73]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"centro_produccion"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard"]]


In [113]:
te_clas[te_clas["nom_produccion"]=="DA2547"]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard
274,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,ENTUBADORA 3,50.0
926,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,ENTUBADORA 2,40.0
1526,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,PLISADORA 3,0.0
2302,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,PLISADORA 4,170.0
2410,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,HENNECKE,225.0
3294,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,BOLSA,0.0
3592,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,MARCADO/HOTMELT/ARANDELA/EMPAQUE,40.0
4639,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,CONTROL DE CALIDAD,30.0
4996,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,EMBALAJE,45.0


In [118]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# 1) Traer Workorders (órdenes de trabajo de producción)
# ===============================

workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line", "search_read",
    [
        [["date_start", ">=", "2023-01-01"], ["state", "=", "done"]],  # domain
        ["id", "name", "production_id", "product_id", "product_internal_ref",
         "workcenter_id", "date_start", "date_finished", "hour",
         "tiempo_optimo_horas", "tiempo_estimado_segundos",
         "tiempo_segundos_hombre", "tiempo_estimado_x_fuerza_lab"]
    ]  # <-- fields como lista, no dict
)

import pandas as pd
df_workorders = pd.DataFrame(workorders)

df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)



In [119]:
df_workorders.sample(0)

,product_internal_ref,hour,date_finished,date_start,tiempo_segundos_hombre,name,tiempo_optimo_horas,production_id,tiempo_estimado_x_fuerza_lab,workcenter_id,tiempo_estimado_segundos,id,product_id,OP


In [121]:
df_workorders[(df_workorders["product_internal_ref"].str.contains("DA2547")) & (df_workorders["OP"].str.contains("MO12844"))]


,product_internal_ref,hour,date_finished,date_start,tiempo_segundos_hombre,name,tiempo_optimo_horas,production_id,tiempo_estimado_x_fuerza_lab,workcenter_id,tiempo_estimado_segundos,id,product_id,OP
4967,DA2547B,11.73,2025-07-28 13:19:31,2025-07-22 09:03:20,33775.0,ENTUBADORA 3 - DA2547 FILTROAIRE 1_ DONSSON -...,9.722222,"[13112, MO12844]",14072.92,"[54, CORTE]",42218.75,76535,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
8104,DA2547B,10.73,2025-07-22 12:54:16,2025-07-22 09:13:29,30912.0,ENTUBADORA 2 - DA2547 FILTROAIRE 1_ DONSSON -...,7.777778,"[13112, MO12844]",38640.00,"[54, CORTE]",38640.00,76536,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
10370,DA2547B,0.00,2025-07-24 14:08:36,2025-07-22 14:07:59,0.0,PLISADORA 3 - DA2547 FILTROAIRE 1_ DONSSON -K...,0.000000,"[13112, MO12844]",0.00,"[56, PLISADO]",0.00,76537,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
13425,DA2547B,21.66,2025-07-23 11:11:59,2025-07-23 06:46:22,124768.0,PLISADORA 4 - DA2547 FILTROAIRE 1_ DONSSON -K...,16.527778,"[13112, MO12844]",38990.00,"[56, PLISADO]",77980.00,76538,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
16812,DA2547B,57.07,2025-07-25 07:14:00,2025-07-24 09:29:44,164360.0,HENNECKE - DA2547 FILTROAIRE 1_ DONSSON -KODI...,43.750000,"[13112, MO12844]",51362.50,"[55, INYECCION]",205450.00,76539,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
20661,DA2547B,0.00,2025-07-25 11:37:48,2025-07-24 16:07:17,0.0,"BOLSA - DA2547 FILTROAIRE 1_ DONSSON -KODIAK,...",0.000000,"[13112, MO12844]",0.00,"[57, EMBALAJE]",0.00,76540,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
24878,DA2547B,10.16,2025-07-28 09:36:12,2025-07-24 13:14:48,29267.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA2547 FIL...,7.777778,"[13112, MO12844]",18291.88,"[57, EMBALAJE]",36583.75,76541,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
29065,DA2547B,9.53,2025-07-28 10:46:11,2025-07-25 08:50:43,27440.0,CONTROL DE CALIDAD - DA2547 FILTROAIRE 1_ DON...,5.833333,"[13112, MO12844]",17150.00,"[57, EMBALAJE]",34300.00,76542,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
32101,DA2547B,11.11,2025-07-28 09:15:29,2025-07-25 09:28:42,32004.0,EMBALAJE - DA2547 FILTROAIRE 1_ DONSSON -KODI...,8.750000,"[13112, MO12844]",13335.00,"[57, EMBALAJE]",40005.00,76543,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DO...",MO12844
